In [1]:
'''Aim to set up test data in the exact same format as final categorical merged train
dataset'''

'Aim to set up test data in the exact same format as final categorical merged train\ndataset'

In [5]:
# Importing modules

import pandas as pd # for data wrangling
import seaborn as sns # for plotting
import matplotlib.pyplot as plt # for plotting
import missingno as msno # for visualizing missing values in data
import numpy as np # for array computations

plt.style.use('ggplot') # setting style for matplotlib

In [15]:
# importing model specific libraries

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score



In [3]:
ls

ChangingCategoricals.ipynb  RunModel.csv.ipynb              train/
FirstLook.ipynb             sample_submissions_wbscxqU.csv  train_ps0mmDv.zip
PrepTestData.ipynb          test_submissions_NeDLEvX.csv


In [4]:
# reading raw test data

test_raw = pd.read_csv("test_submissions_NeDLEvX.csv")

In [5]:
test_raw.head()

,ID,user_id,problem_id
0,user_856_prob_5822,user_856,prob_5822
1,user_2642_prob_2334,user_2642,prob_2334
2,user_2557_prob_2920,user_2557,prob_2920
3,user_1572_prob_4598,user_1572,prob_4598
4,user_295_prob_6139,user_295,prob_6139


In [6]:
cd train

/home/arrayslayer/ML Competitions/Codefest/train


In [7]:
ls

merged_and_categorical.csv  problem_data.csv       user_data.csv
merged_dataset.csv          train_submissions.csv


In [8]:
# importing user properties from training data that had user details
user_data = pd.read_csv('user_data.csv')
problem_data = pd.read_csv("problem_data.csv")
train_df = pd.read_csv("merged_dataset.csv")

In [9]:
# attempting to merge user_df on basis of user_id into test_raw which results in test_user_merge

test_user_merge = pd.merge(test_raw, user_data, on='user_id', how='outer')

In [10]:
# attempting to merge test_user_merge with problem_data on basis of prolem_id to result
# in test_both_merged

test_both_merged = pd.merge(test_user_merge, problem_data, on='problem_id', how='outer')

In [11]:
test_both_merged.head()

,ID,user_id,problem_id,submission_count,problem_solved,contribution,country,follower_count,last_online_time_seconds,max_rating,rating,rank,registration_time_seconds,level_type,points,tags
0,user_856_prob_5822,user_856,prob_5822,274.0,231.0,0.0,India,35.0,1.503234e+09,393.062,362.959,intermediate,1.422163e+09,L,NaN,NaN
1,user_846_prob_5822,user_846,prob_5822,587.0,547.0,2.0,Brazil,47.0,1.505018e+09,423.739,423.739,intermediate,1.434644e+09,L,NaN,NaN
2,user_2183_prob_5822,user_2183,prob_5822,242.0,231.0,0.0,Bolivia,21.0,1.505575e+09,334.862,300.745,beginner,1.463786e+09,L,NaN,NaN
3,user_2553_prob_5822,user_2553,prob_5822,1558.0,1436.0,1.0,Egypt,186.0,1.505553e+09,459.862,455.849,advanced,1.414068e+09,L,NaN,NaN
4,user_856_prob_1868,user_856,prob_1868,274.0,231.0,0.0,India,35.0,1.503234e+09,393.062,362.959,intermediate,1.422163e+09,A,500.0,"implementation,strings"


In [12]:
'''note lots of missing values might be there. Check how to deal with missing values.
Fill with some def value because dropping them would result in not predicting
for them?'''

'note lots of missing values might be there. Check how to deal with missing values.\nFill with some def value because dropping them would result in not predicting\nfor them?'

In [13]:
# beginning with converting strings to numerical categorical variables

In [14]:
'''This function takes in a "dataframe" and a "column_name" in which it has to replace
values. It takes an "original" value and replaces it by the value of the "replacement".'''

def replace_column_values(dataframe, column_name, original, replacement):
    dataframe.replace({column_name:{original : replacement}}, inplace = True)

In [15]:
# invoking replace_column_values to replace values 
# replacing rank. Beginner - 1, intermediate - 2, advanced -3, expert - 4

replace_column_values(test_both_merged,"rank",'beginner',1)
replace_column_values(test_both_merged,"rank",'intermediate',2)
replace_column_values(test_both_merged,"rank",'advanced',3)
replace_column_values(test_both_merged,"rank",'expert',4)

In [16]:
# replacing level_type. A-1,  B-2, ... N - 

alpha = [chr(ch) for ch in range(65,79)]

for i,ch in enumerate(alpha):
    replace_column_values(test_both_merged,'level_type',ch,i+1)

In [17]:
# removing tags

del test_both_merged['tags']

In [18]:
# Should be mapped as same as train data because I used the same train_df with "merged_dataset.csv"
# to get unique country values
# CAREFUL! CHECK IF MAPPED CORRECTLY!
# replacing countries column with numerical values

country_array = train_df['country'].unique()

for i,ch in enumerate(country_array):
    replace_column_values(test_both_merged,'country',ch,i+1)

In [20]:
# sparse points column. How to handle its missing values?

test_both_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68453 entries, 0 to 68452
Data columns (total 15 columns):
ID                           66555 non-null object
user_id                      66625 non-null object
problem_id                   68383 non-null object
submission_count             66625 non-null float64
problem_solved               66625 non-null float64
contribution                 66625 non-null float64
country                      68453 non-null int64
follower_count               66625 non-null float64
last_online_time_seconds     66625 non-null float64
max_rating                   66625 non-null float64
rating                       66625 non-null float64
rank                         66625 non-null float64
registration_time_seconds    66625 non-null float64
level_type                   68049 non-null float64
points                       54456 non-null float64
dtypes: float64(11), int64(1), object(3)
memory usage: 8.4+ MB


In [ ]:
'''Total data is 66555. Some have user_id missing some problem_id missing and 
we take them too, leading to extra data. Deal with it.'''

In [23]:
# saving final merged test dataset that is built on similar grounds as the final 
# training dataset

test_both_merged.to_csv("test_merged.csv")

In [24]:
test_both_merged.head()

,ID,user_id,problem_id,submission_count,problem_solved,contribution,country,follower_count,last_online_time_seconds,max_rating,rating,rank,registration_time_seconds,level_type,points
0,user_856_prob_5822,user_856,prob_5822,274.0,231.0,0.0,2,35.0,1.503234e+09,393.062,362.959,2.0,1.422163e+09,12.0,NaN
1,user_846_prob_5822,user_846,prob_5822,587.0,547.0,2.0,27,47.0,1.505018e+09,423.739,423.739,2.0,1.434644e+09,12.0,NaN
2,user_2183_prob_5822,user_2183,prob_5822,242.0,231.0,0.0,40,21.0,1.505575e+09,334.862,300.745,1.0,1.463786e+09,12.0,NaN
3,user_2553_prob_5822,user_2553,prob_5822,1558.0,1436.0,1.0,3,186.0,1.505553e+09,459.862,455.849,3.0,1.414068e+09,12.0,NaN
4,user_856_prob_1868,user_856,prob_1868,274.0,231.0,0.0,2,35.0,1.503234e+09,393.062,362.959,2.0,1.422163e+09,1.0,500.0


In [28]:
test_both_merged['user_id'].dropna(inplace = True)
test_both_merged['problem_id'].dropna(inplace = True)

In [29]:
test_both_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68453 entries, 0 to 68452
Data columns (total 15 columns):
ID                           66555 non-null object
user_id                      66625 non-null object
problem_id                   68383 non-null object
submission_count             66625 non-null float64
problem_solved               66625 non-null float64
contribution                 66625 non-null float64
country                      68453 non-null int64
follower_count               66625 non-null float64
last_online_time_seconds     66625 non-null float64
max_rating                   66625 non-null float64
rating                       66625 non-null float64
rank                         66625 non-null float64
registration_time_seconds    66625 non-null float64
level_type                   68049 non-null float64
points                       54456 non-null float64
dtypes: float64(11), int64(1), object(3)
memory usage: 8.3+ MB


In [3]:
ls

merged_and_categorical.csv  problem_data.csv  train_submissions.csv
merged_dataset.csv          test_merged.csv   user_data.csv


In [45]:
test_df = pd.read_csv('test_merged.csv')

In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68453 entries, 0 to 68452
Data columns (total 16 columns):
Unnamed: 0                   68453 non-null int64
ID                           66555 non-null object
user_id                      66625 non-null object
problem_id                   68383 non-null object
submission_count             66625 non-null float64
problem_solved               66625 non-null float64
contribution                 66625 non-null float64
country                      68453 non-null int64
follower_count               66625 non-null float64
last_online_time_seconds     66625 non-null float64
max_rating                   66625 non-null float64
rating                       66625 non-null float64
rank                         66625 non-null float64
registration_time_seconds    66625 non-null float64
level_type                   68049 non-null float64
points                       54456 non-null float64
dtypes: float64(11), int64(2), object(3)
memory usage: 8.4+ MB


In [8]:
test_df.columns

Index(['Unnamed: 0', 'ID', 'user_id', 'problem_id', 'submission_count',
       'problem_solved', 'contribution', 'country', 'follower_count',
       'last_online_time_seconds', 'max_rating', 'rating', 'rank',
       'registration_time_seconds', 'level_type', 'points'],
      dtype='object')

In [46]:
# Retaining only those rows that have ID as a legit input
test_df = test_df[pd.notnull(test_df['ID'])]

In [17]:
#importing merged and integral dataset

train_df = pd.read_csv('merged_and_categorical.csv')
train_df.dropna(inplace=True)

In [18]:
# training output

Y_train = train_df['attempts_range']
Y_train = Y_train.astype(int)

In [19]:
# training input dataset

X_train = train_df[['submission_count',
       'problem_solved', 'contribution', 'country', 'follower_count',
       'last_online_time_seconds', 'max_rating', 'rating', 'rank',
       'registration_time_seconds', 'level_type', 'points']]

In [57]:
# impprting test dataset that matches train dataset properties

test_df = pd.read_csv('test_merged.csv')

In [58]:
# Retaining only those rows that have ID as a legit input
test_df = test_df[pd.notnull(test_df['ID'])]

In [69]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66555 entries, 0 to 66554
Data columns (total 16 columns):
Unnamed: 0                   66555 non-null int64
ID                           66555 non-null object
user_id                      66555 non-null object
problem_id                   66555 non-null object
submission_count             66555 non-null float64
problem_solved               66555 non-null float64
contribution                 66555 non-null float64
country                      66555 non-null int64
follower_count               66555 non-null float64
last_online_time_seconds     66555 non-null float64
max_rating                   66555 non-null float64
rating                       66555 non-null float64
rank                         66555 non-null float64
registration_time_seconds    66555 non-null float64
level_type                   66270 non-null float64
points                       54071 non-null float64
dtypes: float64(11), int64(2), object(3)
memory usage: 8.6+ MB


In [59]:
# testing input dataset

X_test = test_df[['submission_count',
       'problem_solved', 'contribution', 'country', 'follower_count',
       'last_online_time_seconds', 'max_rating', 'rating', 'rank',
       'registration_time_seconds', 'level_type', 'points']]

In [22]:
# Training and performing cross validation


rf = RandomForestClassifier(n_estimators=500,max_depth =10, min_samples_split= 4 )
scores = cross_val_score(rf, X_train, Y_train, cv=5, scoring='accuracy')
scores

array([ 0.54457449,  0.54253293,  0.54144348,  0.53577239,  0.54279592])

In [24]:
rf.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=4, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [54]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66555 entries, 0 to 66554
Data columns (total 12 columns):
submission_count             66555 non-null float64
problem_solved               66555 non-null float64
contribution                 66555 non-null float64
country                      66555 non-null int64
follower_count               66555 non-null float64
last_online_time_seconds     66555 non-null float64
max_rating                   66555 non-null float64
rating                       66555 non-null float64
rank                         66555 non-null float64
registration_time_seconds    66555 non-null float64
level_type                   66270 non-null float64
points                       54071 non-null float64
dtypes: float64(11), int64(1)
memory usage: 6.6 MB


In [60]:
#filling in missing values

X_test = X_test.fillna(method='bfill')
X_test = X_test.fillna(value = 0)

In [63]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66555 entries, 0 to 66554
Data columns (total 12 columns):
submission_count             66555 non-null float64
problem_solved               66555 non-null float64
contribution                 66555 non-null float64
country                      66555 non-null int64
follower_count               66555 non-null float64
last_online_time_seconds     66555 non-null float64
max_rating                   66555 non-null float64
rating                       66555 non-null float64
rank                         66555 non-null float64
registration_time_seconds    66555 non-null float64
level_type                   66555 non-null float64
points                       66555 non-null float64
dtypes: float64(11), int64(1)
memory usage: 6.6 MB


In [64]:
# getting predictions

pred = rf.predict(X_test)

In [70]:
# creating submission file


sol = pd.DataFrame({"ID" : test_df['ID'], "attempts_range" : pred})

In [72]:
# saving it

sol.to_csv("submission.csv", index=False)